<a href="https://colab.research.google.com/github/GisstHubana/FHIR_Reporting_Tool/blob/GisstHubana-patch-1/Code/fhir_reporting_tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vorbereitung
CleanUp - Eventuell gespeicherte Variablen löschen vor Ausführung des Colab books für eine bessere Reproduzierbarkeit

In [ ]:
%reset -f

Importieren der notwendigen Libraries

In [ ]:
# import sqlite3

Erstellung einer virtuellen Kopie des GitHub Repositorium FHIR_Reporting_Tool in Colab

In [ ]:
!git clone https://github.com/GisstHubana/FHIR_Reporting_Tool.git

In [ ]:
# Wechsel in den Unterordner csv_data des Github-Projekt-Ordners
# %cd /content/FHIR_Reporting_Tool/ordner

In [ ]:
# Auflistung aller Ordnerinhalte
!ls